# Projeto PicMoney: Análise Visual e Prototipagem do Dashboard (Colab)

**Objetivo:** Este notebook carrega os dados já limpos (`_cleaned.csv`), deriva, integra e recria as principais visualizações e KPIs do dashboard em Python, usando Pandas para agregação e Plotly para visualização.

Isso serve para:
1.  Validar a lógica de dados do backend (`server.js`).
2.  Prototipar visualizações complexas.
3.  Servir como o artefato de análise de dados do projeto.


In [9]:
# Bloco 1: Instalação e Importação de Bibliotecas
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

pd.options.plotting.backend = "plotly"
template_visual = "plotly_dark"

In [12]:
# Bloco 2: Carregar os Dados Limpos
# --- ATENÇÃO ---
# Na barra lateral (ícone de pasta), clique em "Fazer upload"
# e envie os 4 arquivos .csv antes de executar esta célula.

print("Carregando arquivos CSV limpos...")
FILE_TRANSACOES = 'transacoes_cleaned.csv'
FILE_PLAYERS = 'players_cleaned.csv'
FILE_LOJAS = 'lojas_cleaned.csv'
FILE_PEDESTRES = 'pedestres_cleaned.csv'

df_transacoes = pd.DataFrame()
df_players = pd.DataFrame()
df_lojas = pd.DataFrame()
df_pedestres = pd.DataFrame()

try:
    df_transacoes = pd.read_csv(FILE_TRANSACOES)
    df_players = pd.read_csv(FILE_PLAYERS)
    df_lojas = pd.read_csv(FILE_LOJAS)
    df_pedestres = pd.read_csv(FILE_PEDESTRES)

    print("Arquivos carregados com sucesso!")
    print(f"Transações: {len(df_transacoes)} linhas")
    print(f"Players: {len(df_players)} linhas")


    print("\nConvertendo tipos de dados de data...")
    df_transacoes['data'] = pd.to_datetime(df_transacoes['data'])
    df_pedestres['data'] = pd.to_datetime(df_pedestres['data'])
    print("Conversão concluída.")

except FileNotFoundError:
    print("\n--- ERRO CRÍTICO ---")
    print("Arquivos .csv não encontrados. O script não pode continuar.")
    print("Por favor, faça o upload dos 4 arquivos .csv para o Colab (usando o ícone de pasta à esquerda) e rode esta célula novamente.")

except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

Carregando arquivos CSV limpos...
Arquivos carregados com sucesso!
Transações: 100000 linhas
Players: 10000 linhas

Convertendo tipos de dados de data...
Conversão concluída.


### Etapa: Derivar Dados

O passo de "derivação" mais importante após carregar um CSV limpo é garantir que os tipos de dados estejam corretos para a análise. A coluna `data` (em transações e pedestres) foi salva como texto e precisa ser convertida (derivada) de volta para o tipo `datetime`.

In [31]:
# Bloco 3: Derivar Dados (Garantir Tipos Corretos)

if not df_transacoes.empty:
    print("Derivando tipos de dados (Data)...")

    # Converter colunas de data (que são string no CSV) para datetime
    df_transacoes['data'] = pd.to_datetime(df_transacoes['data'])
    df_pedestres['data'] = pd.to_datetime(df_pedestres['data'])

    # Garantir que 'idade' é numérico
    df_players['idade'] = pd.to_numeric(df_players['idade'], errors='coerce').fillna(0)

    print("Tipos de dados derivados com sucesso.")
    print("\nInfo (Transações):")
    df_transacoes.info()
    print("\nInfo (Players):")
    df_players.info()
else:
    print("Dados não carregados. Execute o Bloco 2 primeiro.")

Derivando tipos de dados (Data)...
Tipos de dados derivados com sucesso.

Info (Transações):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   celular                    100000 non-null  int64         
 1   data                       100000 non-null  datetime64[ns]
 2   hora                       100000 non-null  object        
 3   nome_estabelecimento       100000 non-null  object        
 4   bairro_estabelecimento     100000 non-null  object        
 5   categoria_estabelecimento  100000 non-null  object        
 6   id_campanha                100000 non-null  object        
 7   id_cupom                   100000 non-null  object        
 8   tipo_cupom                 100000 non-null  object        
 9   produto                    33328 non-null   object        
 10  valor_cupom             

### Etapa: Integrar os Dados

Este é o passo crucial. Integramos os dados demográficos (`idade`, `sexo`, etc.) do `df_players` ao `df_transacoes` usando a coluna `celular` como chave de junção (merge).

In [32]:
# Bloco 4: Integrar os Dados (Merge)

if not (df_transacoes.empty or df_players.empty):
    print("Iniciando integração (merge) de Transações e Players...")

    # 1. Selecionar apenas as colunas que queremos dos players
    # (Evita colunas duplicadas ou desnecessárias)
    colunas_players = ['celular', 'idade', 'sexo']

    # Verificar se colunas opcionais existem no seu players_cleaned.csv
    if 'Dispositivo' in df_players.columns:
        colunas_players.append('Dispositivo')

    df_players_subset = df_players[colunas_players]

    # 2. Executar a integração (Left Join)
    # Mantém todas as transações e adiciona dados do player, se encontrado.
    df_transacoes_integradas = pd.merge(
        df_transacoes,
        df_players_subset,
        on='celular',
        how='left'
    )

    # 3. Pós-processamento (Limpeza do Merge)
    # Se uma transação não encontrar um player, preenchemos com 'Não informado'
    df_transacoes_integradas['sexo'] = df_transacoes_integradas['sexo'].fillna('Não informado')
    df_transacoes_integradas['idade'] = df_transacoes_integradas['idade'].fillna(0)
    if 'Dispositivo' in df_transacoes_integradas.columns:
        df_transacoes_integradas['Dispositivo'] = df_transacoes_integradas['Dispositivo'].fillna('Não informado')

    print("Integração concluída com sucesso!")
    print(f"Total de linhas nas transações integradas: {len(df_transacoes_integradas)}")

    print("\nAmostra dos dados integrados (verificando colunas 'idade' e 'sexo'):")
    print(df_transacoes_integradas[['celular', 'valor_cupom', 'idade', 'sexo']].head())

else:
    print("Dados não carregados. Execute os blocos anteriores primeiro.")

Iniciando integração (merge) de Transações e Players...
Integração concluída com sucesso!
Total de linhas nas transações integradas: 100011

Amostra dos dados integrados (verificando colunas 'idade' e 'sexo'):
       celular  valor_cupom  idade       sexo
0  61964978673       229.64     30      Outro
1  11942316424       356.33     77  Masculino
2  11979652178       719.06     55  Masculino
3  11934184646       798.34     78   Feminino
4  11979731725       718.45     78  Masculino


### Etapa: Formatar os Dados

Salvamos o resultado da integração (`df_transacoes_integradas`) em um novo arquivo CSV.

In [34]:
# Bloco 5: Formatar os Dados (Exportar resultado)

if 'df_transacoes_integradas' in locals():

    # Define o nome do arquivo de saída
    output_final = 'transacoes_final_integrada.csv'

    # Salvar
    df_transacoes_integradas.to_csv(output_final, index=False, sep=',', encoding='utf-8')

    print("\n--- Processo Concluído ---")
    print(f"Arquivo final formatado e salvo como: {output_final}")
    print("Você pode baixar este arquivo (pelo ícone de pasta à esquerda) e usá-lo no seu backend.")

else:
    print("A integração não foi executada. Rode o Bloco 4 primeiro.")


--- Processo Concluído ---
Arquivo final formatado e salvo como: transacoes_final_integrada.csv
Você pode baixar este arquivo (pelo ícone de pasta à esquerda) e usá-lo no seu backend.


## Visão 1: KPIs Gerais (Cálculo)
(Ref: Rota `/api/general-stats`)

In [28]:
# Bloco 6: Cálculo dos KPIs Gerais

# Verificar se os dados foram carregados antes de continuar
if not df_transacoes.empty:
    receita_total = df_transacoes['valor_cupom'].sum()
    comissao_total = df_transacoes['repasse_picmoney'].sum()
    total_transacoes = len(df_transacoes)
    ticket_medio = receita_total / total_transacoes
    clientes_unicos = df_transacoes['celular'].nunique()
    lojas_unicas = df_transacoes['nome_estabelecimento'].nunique()

    print("--- KPIs Gerais ---")
    print(f"Receita Bruta Total: R$ {receita_total:,.2f}")
    print(f"Comissão PicMoney: R$ {comissao_total:,.2f}")
    print(f"Total de Transações: {total_transacoes:,}")
    print(f"Ticket Médio: R$ {ticket_medio:,.2f}")
    print(f"Clientes Únicos: {clientes_unicos:,}")
    print(f"Lojas Únicas: {lojas_unicas:,}")
else:
    print("Dados de transações não carregados. Pule esta célula ou corrija o Bloco 2.")

--- KPIs Gerais ---
Receita Bruta Total: R$ 55,048,958.53
Comissão PicMoney: R$ 7,047,473.91
Total de Transações: 100,000
Ticket Médio: R$ 550.49
Clientes Únicos: 4,813
Lojas Únicas: 33


## Visão 2: Receita ao Longo do Tempo
(Ref: Rota `/api/transactions-over-time` | Componente: `LineChart.jsx`)

In [15]:
# Bloco 7: Gráfico de Linha - Receita ao Longo do Tempo

if not df_transacoes.empty:
    df_time = df_transacoes.groupby('data').agg(
        receita_total=('valor_cupom', 'sum'),
        comissao_picmoney=('repasse_picmoney', 'sum'),
        transacoes=('id_cupom', 'count')
    ).reset_index()

    fig = px.line(df_time, x='data', y=['receita_total', 'comissao_picmoney'],
                  title='Receita Total vs. Comissão PicMoney ao Longo do Tempo',
                  labels={'value': 'Valor (R$)', 'data': 'Data', 'variable': 'Métrica'},
                  template=template_visual)

    fig.update_layout(hovermode="x unified")
    fig.show()
else:
    print("Dados de transações não carregados.")


## Visão 3: Distribuição de Cupons
(Ref: Rota `/api/coupon-distribution` | Componente: `DonutChart.jsx`)

In [16]:
# Bloco 8: Gráfico de Pizza/Donut - Distribuição de Cupons

if not df_transacoes.empty:
    df_coupon_dist = df_transacoes.groupby('tipo_cupom').agg(
        quantidade=('id_cupom', 'count')
    ).reset_index()

    fig = px.pie(df_coupon_dist, names='tipo_cupom', values='quantidade',
                 title='Distribuição de Cupons Capturados por Tipo',
                 hole=0.4, # Define o "buraco" no meio para virar um Donut Chart
                 template=template_visual)

    fig.update_traces(textinfo='percent+label', pull=[0.05, 0.05, 0.05])
    fig.show()
else:
    print("Dados de transações não carregados.")

## Visão 4: Top Categorias e Lojas
(Ref: Rota `/api/top-categories` | Componente: `BarChart.jsx`)

In [17]:
# Bloco 9: Gráfico de Barras - Top 10 Categorias

if not df_transacoes.empty:
    df_top_cat = df_transacoes.groupby('categoria_estabelecimento').agg(
        valor=('valor_cupom', 'sum')
    ).reset_index().sort_values('valor', ascending=False).head(10)

    fig = px.bar(df_top_cat, x='valor', y='categoria_estabelecimento',
                 orientation='h',
                 title='Top 10 Categorias por Receita Gerada',
                 labels={'valor': 'Receita Total (R$)', 'categoria_estabelecimento': 'Categoria'},
                 template=template_visual)

    # Ordena o eixo Y para mostrar a maior barra no topo
    fig.update_layout(yaxis={'categoryorder':'total ascending'})
    fig.show()
else:
    print("Dados de transações não carregados.")

## Visão 5: Análise Geográfica
(Ref: Rota `/api/geographic/pedestres-heatmap` | Componente: `GeographicHeatmap.jsx`)

In [18]:
# Bloco 10: Mapa de Calor - Densidade de Pedestres

if not df_pedestres.empty:
    fig = px.density_mapbox(df_pedestres, lat='latitude', lon='longitude',
                           radius=8,
                           center=dict(lat=-23.563, lon=-46.57), # Centro da Av. Paulista
                           zoom=13,
                           mapbox_style="open-street-map", # Não requer token
                           title='Heatmap de Densidade de Pedestres (Av. Paulista)')

    fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})
    fig.show()
else:
    print("Dados de pedestres não carregados.")

## Visão 6: Horários de Pico
(Ref: Rota `/api/time-analysis/peak-hours` | Componente: `HeatmapChart.jsx`)

In [19]:
# Bloco 11: Heatmap - Horários de Pico de Transações

if not df_transacoes.empty:

    # --- CORREÇÃO DESTE BLOCO ---
    # Derivar dia da semana da coluna 'data' (que já é datetime)
    df_transacoes['dia_semana'] = df_transacoes['data'].dt.day_name()

    # Derivar hora da coluna 'hora' (que é string 'HH:MM:SS')
    # Convertemos para datetime e extraímos a hora
    df_transacoes['hora_dia'] = pd.to_datetime(df_transacoes['hora'], format='%H:%M:%S').dt.hour
    # --- FIM DA CORREÇÃO ---

    # Criar a matriz de pivot
    df_peak = df_transacoes.groupby(['dia_semana', 'hora_dia']).agg(
        transacoes=('id_cupom', 'count')
    ).reset_index()

    pivot = df_peak.pivot(index='dia_semana', columns='hora_dia', values='transacoes').fillna(0)

    # Reordenar os dias da semana
    dias_ordem = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    pivot = pivot.reindex(dias_ordem)

    # Criar o gráfico
    fig = go.Figure(data=go.Heatmap(
                        z=pivot.values,
                        x=[f"{h}:00" for h in pivot.columns], # Formatar eixo X
                        y=pivot.index,
                        colorscale='Viridis')) # Escala de cor

    fig.update_layout(title='Heatmap de Transações por Dia da Semana e Hora',
                      xaxis_title='Hora do Dia',
                      yaxis_title='Dia da Semana',
                      template=template_visual)
    fig.show()
else:
    print("Dados de transações não carregados.")

## Visão 7: Análise de Usuários (Demografia)
(Ref: Rota `/api/customer-segments` | Componente: `DemographicPyramid.jsx` ou `ScatterChart.jsx`)

In [22]:
# Bloco 12: Gráfico de Barras - Contagem de Players por Faixa Etária e Gênero

if not df_players.empty:
    # 1. Filtrar dados inválidos
    df_players_valid = df_players[
        (df_players['sexo'] != 'Não informado') &
        (df_players['idade'] > 0)
    ]

    # 2. Criar faixas etárias (bins)
    bins = [0, 18, 25, 35, 45, 55, 65, 100]
    labels = ['0-18', '19-25', '26-35', '36-45', '46-55', '56-65', '65+']
    df_players_valid['faixa_etaria'] = pd.cut(df_players_valid['idade'], bins=bins, labels=labels, right=True)

    # 3. Agregar os dados
    df_demographic = df_players_valid.groupby(['faixa_etaria', 'sexo']).agg(
        quantidade=('celular', 'count')
    ).reset_index()

    # 4. Plotar o Gráfico de Barras Agrupado
    fig = px.bar(df_demographic,
                 x='faixa_etaria',
                 y='quantidade',
                 color='sexo',
                 barmode='group', # 'group' = barras lado a lado
                 title="Contagem de Players por Faixa Etária e Gênero",
                 labels={'faixa_etaria': 'Faixa Etária', 'quantidade': 'Número de Players', 'sexo': 'Gênero'},
                 template=template_visual)

    fig.show()
else:
    print("Dados de players não carregados.")

/tmp/ipython-input-765872367.py:16: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

